In [133]:
#import modules
import numpy as np
import scipy.stats as sts
import scipy.optimize as opt
import scipy.special as spc
import scipy.integrate as integrate
import pandas as pd

In [134]:
data = pd.read_csv('MacroSeries.txt',sep=',',header=None,names=['c_t','k_t','w_t','r_t'])

In [135]:
#part a
#from equation 3,
def z_t(w_t,k_t,alpha):
    return np.log(w_t/((1-alpha) * k_t ** alpha))    


#from equation 5
def eps_t(z_t, rho, mu):
    z_t_1=np.append(mu, z_t[:-1])
    return z_t - rho * z_t_1 - (1-rho) * mu


# Define normal distribution and the likelihood function
def norm_pdf(w_t, k_t, alpha, rho, mu, sigma):
    eps_vals = eps_t(z_t(w_t,k_t,alpha), rho, mu)
    pdf_vals    = np.maximum((1/np.sqrt(2 * np.pi * sigma**2)) * np.exp( - (eps_vals)**2 / (2 * sigma**2)), 1e-20)
    return pdf_vals
                    
def log_lik_normal(w_t, k_t, alpha, rho, mu, sigma):
    pdf_vals = norm_pdf(w_t, k_t, alpha, rho, mu, sigma)
    ln_pdf_vals = np.log(pdf_vals)
    log_lik_val = ln_pdf_vals.sum()
    return log_lik_val


# Define Objective Function
def normcrit(params, w_t, k_t):
    alpha, rho, mu, sigma = params
    log_lik_val = log_lik_normal(w_t,k_t, alpha, rho, mu, sigma)
    return -log_lik_val

In [136]:
#optimize the result
#randomly guess for initial values
alpha_0, rho_0, mu_0, sigma_0 = 0.4, 0.7, 10, 1
params_init = np.array([alpha_0, rho_0, mu_0, sigma_0])
mle1_args = (data ["w_t"], data["k_t"])
results1 = opt.minimize(normcrit, params_init, args=(mle1_args), method='L-BFGS-B',bounds = ((1e-8, 1-1e-8), (-1+1e-8, 1-1e-8), (1e-8, None), (1e-8, None)))
alpha_MLE, rho_MLE, mu_MLE, sigma_MLE = results1.x
hessinv1 = results1.hess_inv

#get MLE results
print('alpha_MLE=', alpha_MLE, ' rho_MLE=', rho_MLE, 'mu_MLE=', mu_MLE, ' sigma_MLE=', sigma_MLE)
print("log likelikehood=", -results1.fun)


alpha_MLE= 0.457518425402  rho_MLE= 0.720486871779 mu_MLE= 9.52269036866  sigma_MLE= 0.0919962297507
log likelikehood= 96.70690807310774


In [137]:
#get variance covariance matrix
vcv_mle1 = results1.hess_inv.todense()
print('VCV(MLE) = ', vcv_mle1)

VCV(MLE) =  [[  2.59146358e+02  -9.88010739e+01  -3.62342544e+03   1.25824998e+01]
 [ -9.88010739e+01   4.32793330e+01   1.38268318e+03  -5.18531620e+00]
 [ -3.62342544e+03   1.38268318e+03   5.06635800e+04  -1.76015963e+02]
 [  1.25824998e+01  -5.18531620e+00  -1.76015963e+02   6.38087660e-01]]


In [138]:
#part b
#from equation 4,
def z_tt(r_t,k_t,alpha):
    return np.log(r_t/((alpha) * k_t ** (alpha-1)))    


#from equation 5
def eps_t(z_tt, rho, mu):
    z_tt_1=np.append(mu, z_tt[:-1])

    return z_tt - rho * z_tt_1 - (1-rho) * mu

# Define normal distribution and the likelihood function
def norm_pdf(r_t, k_t, alpha, rho, mu, sigma):
    eps_vals = eps_t(z_tt(r_t,k_t,alpha), rho, mu)
    pdf_vals    = np.maximum((1/np.sqrt(2 * np.pi * sigma**2)) * np.exp( - (eps_vals)**2 / (2 * sigma**2)), 1e-20)
    return pdf_vals
                    
def log_lik_normal(r_t, k_t, alpha, rho, mu, sigma):
    epsvals = eps_t(z_tt(r_t,k_t,alpha), rho, mu)
    pdf_vals = norm_pdf(r_t, k_t, alpha, rho, mu, sigma)
    ln_pdf_vals = np.log(pdf_vals)
    log_lik_val = ln_pdf_vals.sum()

    return log_lik_val


# Define Objective Function
def normcrit(params, r_t, k_t):
    alpha, rho, mu, sigma = params
    log_lik_val = log_lik_normal(r_t, k_t, alpha, rho, mu, sigma)
    return -log_lik_val


In [139]:
#optimize the result
#randomly guess for initial values
alpha_0, rho_0, mu_0, sigma_0 = 0.4, 0.7, 10, 1
params_init = np.array([alpha_0, rho_0, mu_0, sigma_0])
mle1_args = (data ["r_t"], data["k_t"])
results2 = opt.minimize(normcrit, params_init, args=(mle1_args), method='L-BFGS-B',bounds = ((1e-8, 1-1e-8), (-1+1e-8, 1-1e-8), (1e-8, None), (1e-8, None)))
alpha_MLE, rho_MLE, mu_MLE, sigma_MLE = results2.x
hessinv1 = results2.hess_inv

#get MLE results
print('alpha_MLE=', alpha_MLE, ' rho_MLE=', rho_MLE, 'mu_MLE=', mu_MLE, ' sigma_MLE=', sigma_MLE)
print("log likelikehood=", -results2.fun)

alpha_MLE= 0.457477871182  rho_MLE= 0.720517319091 mu_MLE= 9.37098406761  sigma_MLE= 0.0919962149614
log likelikehood= 96.70690802228295


In [140]:
#get variance covariance matrix
vcv_mle2 = results2.hess_inv.todense()
print('VCV(MLE) = ', vcv_mle2)

VCV(MLE) =  [[  2.53128754e+01   1.70127003e+01  -4.47213824e+02  -1.01760191e+00]
 [  1.70127003e+01   1.51015454e+01  -3.01117296e+02  -2.14062313e-01]
 [ -4.47213824e+02  -3.01117296e+02   7.90122283e+03   1.79028815e+01]
 [ -1.01760191e+00  -2.14062313e-01   1.79028815e+01   1.03233016e-01]]


In [141]:
#part c
z_crit = np.log(1)-np.log(alpha_MLE)+(1-alpha_MLE)*np.log(7500000)
print ('zcrit =', z_crit)

def norm_pdf_z_crit(z_t, rho, sigma, mu):
    pdf_vals    = np.maximum((1/np.sqrt(2 * np.pi * sigma**2)) * np.exp( - (z_t-rho*10-(1-rho)*mu)**2 / (2 * sigma**2)), 1e-20)
    return pdf_vals

probability_r_1 = 1 - integrate.quad(lambda x: norm_pdf_z_crit(x, rho_MLE, sigma_MLE, mu_MLE), 0, z_crit)[0]
print('Probability that the interest rate at t will be greater than 1:', probability_r_1)

zcrit = 9.37037643911
Probability that the interest rate at t will be greater than 1: 0.9999995952808879
